### Обучите модель, которая предскажет время задержки авиарейсов.

In [65]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 

#### Подготовка данных
Разберём задачу, загрузим данные и изучим признаки.
Нагрузка на инфраструктуру авиалиний растёт с каждым годом. Например, в декабре 2018 года международный аэропорт Мумбаи (бывший Бомбей, Индия) установил мировой рекорд: за сутки взлёт и посадку выполнили 1007 самолётов! 
Задержка рейса вносит беспорядок в работу аэропорта. И приносит убытки ему и авиакомпании.
Решая задачу регрессии, обучим модель, которая спрогнозирует задержку рейса в минутах.
чёрно-белая иллюстрация: на взлётной полосе тюлень. Девушка читает сообщение, что рейс задержан из-за тюленя.
Описание данных
Признаки, с которыми будем работать:
    
Month — месяц авиаперелёта

Day — день авиаперелёта

Day Of Week — день недели перелёта

Airline — название авиакомпани

Origin Airport Delay Rate — частота задержек рейсов из аэропорта вылета

Destination Airport Delay Rate — частота задержек рейсов в аэропорт прилёта

Scheduled Time — запланированное время в пути

Distance — дальность полёта

Scheduled Departure Hour — запланированное время вылета (час)

Scheduled Departure Minute — запланированное время вылета (минута)

#### Целевой признак

Arrival Delay — задержка прилёта (минуты)

In [4]:
# Загрузите данные из файла flights.csv.
# Напечатайте на экране:
# размер таблицы;
# первые пять строк таблицы.
# Изучите данные.
df = pd.read_csv('flights.csv')

In [7]:
df.shape

(77909, 11)

In [8]:
df.head(5)

,Month,Day,Day Of Week,Airline,Origin Airport Delay Rate,Destination Airport Delay Rate,Scheduled Time,Distance,Scheduled Departure Hour,Scheduled Departure Minute,Arrival Delay
0,1,2,5,WN,0.000000,0.000000,225,1390,8,55,-13.0
1,1,4,7,AA,0.000000,0.590909,172,1096,8,30,-12.0
2,1,4,7,US,0.000000,0.000000,46,130,0,30,189.0
3,1,3,6,WN,0.000000,0.000000,160,925,9,25,-7.0
4,1,5,1,EV,0.633333,0.000000,69,113,5,46,-4.0


Преобразуйте категориальный признак техникой OHE. 

Приведите численные признаки к одному масштабу. 

Напечатайте на экране размеры таблиц (уже в прекоде).

In [11]:
df.dtypes

Month                               int64
Day                                 int64
Day Of Week                         int64
Airline                            object
Origin Airport Delay Rate         float64
Destination Airport Delay Rate    float64
Scheduled Time                      int64
Distance                            int64
Scheduled Departure Hour            int64
Scheduled Departure Minute          int64
Arrival Delay                     float64
dtype: object

In [12]:
print(pd.get_dummies(df['Airline'].head(5)))

   AA  EV  US  WN
0   0   0   0   1
1   1   0   0   0
2   0   0   1   0
3   0   0   0   1
4   0   1   0   0


Приведите численные признаки к одному масштабу. Напечатайте на экране размеры таблиц (уже в прекоде).

In [19]:
#< преобразуйте данные так, чтобы избежать дамми-ловушки >
data_ohe = pd.get_dummies(df, drop_first=True)
 
 
target = data_ohe['Arrival Delay']
features = data_ohe.drop(['Arrival Delay'] , axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(
 features, target, test_size=0.25, random_state=12345)
 
    
numeric = ['Day', 'Day Of Week', 'Origin Airport Delay Rate',
       'Destination Airport Delay Rate', 'Scheduled Time', 'Distance',
       'Scheduled Departure Hour', 'Scheduled Departure Minute']
 
scaler = StandardScaler()
scaler.fit(features_train[numeric],)

# < преобразуйте валидационную выборку >
features_valid[numeric] = scaler.transform(features_valid[numeric])
 
print(features_train.shape)
print(features_valid.shape)

(58431, 22)
(19478, 22)


C:\Users\radim\anaconda3\envs\ML_STEPIC\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\radim\anaconda3\envs\ML_STEPIC\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [28]:
# Загрузите данные из файла /datasets/flights_preprocessed.csv.
# Обучите линейную регрессию.
# Посчитайте значение MSE на валидационной выборке и сохраните его в переменной mse. 
# Напечатайте на экране значения MSE и RMSE (уже в прекоде).
df = pd.read_csv('flights_preprocessed.csv')

In [31]:
X = df.drop('Arrival Delay',axis=1)

In [33]:
y = df['Arrival Delay']

In [35]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=12345)

In [39]:
model_lr = LinearRegression()
model_lr.fit(X_train,y_train)

LinearRegression()

In [41]:
pred_test = model_lr.predict(X_test)

In [42]:
mse = mean_squared_error(y_test,pred_test)

In [46]:
print(f'MSE: {mse}')
print(f'RMSE: {mse**0.5}')

MSE: 2129.8240528555293
RMSE: 46.1500168240005


In [47]:
# Вычислите MSE и RMSE для константной модели: каждому объекту она прогнозирует среднее значение целевого признака.
# Сохраните её предсказания в переменной predicted_valid. 
# Напечатайте на экране значения MSE и RMSE (уже в прекоде)
predicted_valid = pd.Series(y_train.mean(), index=y_test.index)
mse = mean_squared_error(y_test,predicted_valid ) 

In [50]:
print("Mean")
print(f"MSE = {mse}")
print(f"RMSE = {mse ** 0.5}")

Mean
MSE = 2358.8874869200226
RMSE = 48.568379496540985


### Коэффициент детерминации
Чтобы не пришлось всё время сравнивать модель со средним, введём новую метрику. Она выражена не в абсолютных значениях, а в относительных.
Коэффициент детерминации, или метрика R2 (англ. coefficient of determination; R-squared), вычисляет долю средней квадратичной ошибки модели от MSE среднего, а затем вычитает эту величину из единицы.
#### Увеличение метрики означает прирост качества модели. 
Формула расчёта R2 выглядит так:

R2 = 1 - MSE модели / MSE среднего

Значение метрики R2 равно единице только в одном случае, если MSE нулевое. 
Такая модель предсказывает все ответы идеально.

R2 равно нулю: модель работает так же, как и среднее.

Если метрика R2 отрицательна, качество модели очень низкое.
#### Значения R2 больше единицы быть не может.

In [57]:
# Вычислите для линейной регрессии значение R2.
# Найдите в документации sklearn.metrics функцию для подсчёта этой метрики. 
# Импортируйте её. 
# Напечатайте результат на экране (уже в прекоде).
# from sklearn.metrics import r2_score 
df_r2 = pd.read_csv('flights_preprocessed.csv')

In [59]:
X = df_r2.drop('Arrival Delay',axis=1)
y = df_r2['Arrival Delay']

In [60]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=32)

In [62]:
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [63]:
predicted_valid = model.predict(X_test)

In [64]:
print(f'R2={r2_score(y_test,predicted_valid)}')

R2=0.09961145060220278


#### Вы найдёте модель с наибольшим значением R2. 

Поэкспериментируйте в Jupyter Notebook и доведите эту метрику до 0.14.  

In [72]:
data = pd.read_csv('flights_preprocessed.csv')

target = data['Arrival Delay']
features = data.drop(['Arrival Delay'] , axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=34)

In [73]:
model = LinearRegression()
model.fit(features_train, target_train)
print(model.score(features_valid, target_valid))

0.10231294291475224


In [85]:
%%time

best_model = None
best_result = 0
for depth in range(1,16,1):
    model = RandomForestRegressor(n_estimators=30, max_depth=depth,random_state=34)
    model.fit(features_train,target_train)
    result = model.score(features_valid,target_valid)
    if result>best_result:
        best_model = model
        best_result = result
        depth_model = depth
print(f"Accuracy наилучшей модели на валидационной выборке: {best_result}",\
      f'Количество деревьев {depth_model}')

Accuracy наилучшей модели на валидационной выборке: 0.17518341571423668 Количество деревьев 11
Wall time: 24min 29s


In [86]:
%%time

model = RandomForestRegressor(n_estimators=60, max_depth= 11, random_state=12345)
model.fit(features_train, target_train)
print(model.score(features_train, target_train))
print(model.score(features_valid, target_valid))


0.393672671430619
0.16919868955541173
Wall time: 11.9 s
